In [28]:
#
# Inspired by https://www.kaggle.com/lpkirwin/fivethirtyeight-elo-ratings
#

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('./data/RegularSeasonCompactResults.csv')

In [29]:
df2017 = df.loc[df['Season'] == 2017]

In [30]:
team_ids = set(df2017.WTeamID).union(set(df2017.LTeamID))
len(team_ids)

351

In [31]:
from glicko2 import Player

In [32]:
def glicko_rounds(glicko, preds, df):
    for row in df2017.itertuples():
        w = row.WTeamID
        l = row.LTeamID

        w_rating, l_rating = glicko[w].getRating(), glicko[l].getRating()
        w_rd, l_rd = glicko[w].getRd(), glicko[l].getRd()

        #
        # We are predicting if the team with the lower ID wins.
        # Glicko would predict the lower ID team to win if their glicko score was higher. 
        #
        if w_rating > l_rating:
            preds.append(1.)
        else:
            preds.append(0.)

        glicko[w].update_player([l_rating], [l_rd], [1])
        glicko[l].update_player([w_rating], [w_rd], [0])

In [33]:
glicko = dict(zip(list(team_ids), [Player() for _ in range(len(team_ids))]))
preds = []
glicko_rounds(glicko, preds, df2017)
sum(preds) / len(preds)  # Prediction accuracy of Glicko2 in regular season. 

0.6646895273401298

In [34]:
preds = []
df = pd.read_csv('./data/ConferenceTourneyGames.csv')
df2017 = df.loc[df['Season'] == 2017]
glicko_rounds(glicko, preds, df2017)
sum(preds) / len(preds)  # Accuracy of Glicko2 in Conference tourney, conditioned on regular season.

0.8129251700680272

In [35]:
preds = []
df = pd.read_csv('./data/NCAATourneyCompactResults.csv')
df2017 = df.loc[df['Season'] == 2017]
glicko_rounds(glicko, preds, df2017)
sum(preds) / len(preds)  # Accuracy of Glicko2 in NCAA tourney, conditioned on regular season and conf. tourney. 

0.6417910447761194